In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
#stopwords.remove("not")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wn = nltk.WordNetLemmatizer()
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('/content/FinalData.csv')

In [3]:
df.sample(frac=1).head()

,Label,Description
113,ai,Google is bringing its ChatGPT-style chatbot B...
325,economics,E. Tammy Kim on the Japanese philosopher Kohei...
257,technology,The Democratic senator from Hawaii explains wh...
34,recession,Strategists say investors should still âstay...
79,recession,"Stocks are poised to disappoint, recession ala..."


In [4]:
df.isnull().sum()

Label          0
Description    2
dtype: int64

In [5]:
df.shape

(396, 2)

In [6]:
df.dropna(inplace=True)

## Data Cleaning : Text Preprocessing

###Remove punctuations, stopwords
---



In [7]:
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    return text

def remove_stopwords(text):
    text = [word for word in text if word not in STOPWORDS]
    return text

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

In [8]:
df['quote_punct_removed'] = df['Description'].apply(lambda x: remove_punct(x))
df["quote_tokenized"] = df["Description"].apply(lambda x: tokenization(x))
df["quote_stop_word"] = df["quote_tokenized"].apply(lambda x: remove_stopwords(x))
df["quotes_cleaned"] = df["quote_stop_word"].apply(lambda x: " ".join(x))

In [9]:
df.head(2)

,Label,Description,quote_punct_removed,quote_tokenized,quote_stop_word,quotes_cleaned
0,recession,"""The longer rates stay high, the more the Fed ...",The longer rates stay high the more the Fed ri...,"[, The, longer, rates, stay, high, the, more, ...","[, The, longer, rates, stay, high, Fed, risks,...",The longer rates stay high Fed risks seriousl...
1,recession,With the economy looming large in the 2024 ele...,With the economy looming large in the electio...,"[With, the, economy, looming, large, in, the, ...","[With, economy, looming, large, 2024, election...",With economy looming large 2024 election Treas...


In [10]:
texts = []
texts = df['quotes_cleaned'].to_list()

In [11]:
len(texts)

394

## Training the word2vec model

In [12]:
from gensim.models import Word2Vec

In [13]:
sentences = [line.split() for line in texts]

w2v =Word2Vec(sentences, window=5, workers=4, min_count=5)

In [14]:
print(sentences[20:25])

[['The', 'US', 'probably', 'tip', 'economic', 'downturn', 'end', 'year', 'Wells', 'Fargo', 'says'], ['Ahead', 'release', 'book', 'Flip', 'Your', 'Life', 'Tarek', 'El', 'Moussa', 'spoke', 'Business', 'Insider', 'state', 'real', 'estate', 'California'], ['US', 'economic', 'growth', 'beat', 'expectations', 'fourth', 'quarter', 's', 'outpacing', 'world', 's', 'wealthiest', 'countries'], ['There', 's', 'powerful', 'seductive', 'narrative', 'soft', 'landing', 'seeing', 'data'], ['From', 'mergers', 'AI', 'EU', 's', 'aggressive', 'rule', 'making', 'hampers', 'ability', 'compete', 'China', 'U', 'S', 'Greg', 'Ip', 'writing', 'WSJ', 'These', 'humbling', 'times', 'Europe', 'The', 'continent', 'barely', 'escaped', 'recession', 'late', 'year', 'U', 'S', 'boomed', 'It', 'losing', 'ouâ']]


In [15]:
words = list(w2v.wv.index_to_key)

In [23]:
words[0:55]

['s',
 'The',
 'AI',
 'â',
 'new',
 'A',
 'year',
 'said',
 'US',
 't',
 'Google',
 'generative',
 'Apple',
 'company',
 'In',
 'S',
 'data',
 'economy',
 'says',
 '2024',
 'like',
 'It',
 'users',
 'I',
 'big',
 'announced',
 'called',
 'world',
 'recession',
 'time',
 'OpenAI',
 '3',
 'technology',
 'chatbot',
 'according',
 'companies',
 'better',
 '2',
 'This',
 'If',
 'U',
 'market',
 'week',
 'feature',
 'days',
 'rates',
 'Pro',
 'Microsoft',
 'years',
 'use',
 'app',
 'features',
 'future',
 'half',
 'Meta']

In [17]:
print(len(words))

232


### How the word vector looks like

In [19]:
print( w2v.wv['Apple'] )

[-4.16924851e-03 -1.62502704e-03  9.89177264e-03 -7.03477114e-03
  5.35563938e-03 -3.55950720e-03  1.94074644e-03  1.91612728e-02
  6.03737496e-03 -8.67814198e-03 -5.86200831e-03 -1.12480884e-02
 -1.02375653e-02  2.26519143e-04 -6.58311136e-03 -1.05463965e-02
  1.64234149e-03 -2.36468064e-03  4.58799582e-03 -1.43743316e-02
 -4.89333970e-03  2.31238897e-03  1.86332280e-03  6.55245176e-03
  2.90973927e-04  6.59807865e-03 -1.04075745e-02  6.94767060e-03
 -8.11570790e-03  9.14044213e-03 -2.38293153e-03  3.95667553e-03
  5.16681280e-03 -1.68531504e-03 -1.02607319e-02 -1.18060515e-03
 -5.17949834e-03 -1.17476033e-02 -1.92888686e-03 -8.65022372e-03
 -3.17607331e-03  1.72284443e-03 -3.27117415e-03 -3.19913868e-03
 -3.55345686e-03 -9.42217465e-03  6.50751451e-03 -5.95593825e-03
 -2.30785203e-03  8.18432495e-03  8.18588398e-03 -8.82035121e-03
  1.96489063e-03 -6.70998963e-03  3.50949733e-04  4.54903551e-04
 -3.57554783e-03  4.05698374e-04  3.91851179e-03  9.15840734e-03
 -8.80792551e-03 -5.02335

### Check similarity scores between words

In [22]:
w2v.wv.similarity('generative', 'OpenAI')

0.5528286

In [25]:
w2v.wv.similarity('recession', 'economy')

0.057826098

### Check for similar words

In [30]:
print(w2v.wv.most_similar('recession'))

[('s', 0.5396994948387146), ('year', 0.5118488669395447), ('early', 0.449627548456192), ('called', 0.44142916798591614), ('2024', 0.40057632327079773), ('AI', 0.39948153495788574), ('data', 0.3913474678993225), ('OpenAI', 0.3900616765022278), ('app', 0.37603759765625), ('â', 0.3758564591407776)]


### Similarly, we can use the same function to find analogies of the form: if x:y, then z:?. Here we enter the known relation x,y in the positive parameter, and the term who’s analogy has to be found in the negative parameter.

In [31]:
print(w2v.wv.most_similar(positive=['Apple', 'company'], negative=['chatbot']))

[('new', 0.4516613781452179), ('2', 0.42468684911727905), ('s', 0.38788115978240967), ('OpenAI', 0.3867313265800476), ('The', 0.38657113909721375), ('â', 0.38648664951324463), ('looks', 0.37457913160324097), ('Netflix', 0.3687882125377655), ('deal', 0.35244229435920715), ('Federal', 0.35118046402931213)]
